In [3]:
import akshare as ak

# 获取所有基金名称和代码
fund_data = ak.fund_name_em()

# 筛选名称中包含纳斯达克字样的基金
stock_list = fund_data[fund_data['基金简称'].str.contains('纳斯达克') | fund_data['基金简称'].str.contains('纳指')]

# 打印包含纳斯达克字样的基金
stock_list = stock_list['基金代码']
display(stock_list)

39       000055
590      000834
2590     003722
4498     006479
4499     006480
          ...  
20684    513290
20685    513300
20692    513390
20716    513870
21283    539001
Name: 基金代码, Length: 80, dtype: object

In [1]:
# import pandas as pd
# # 获取基金持仓数据，参数为基金代码和年份
# fund_code = "159509"  # 替换为你感兴趣的基金代码
# date = 2024  # 替换为你感兴趣的年份
# fund_holdings = ak.fund_portfolio_hold_em(symbol= fund_code, date= date)
# # fund_holdings = ak.stock_fund_stock_holder(symbol= "513100")
# display(fund_holdings)

# df1 = ak.fund_individual_detail_hold_xq(symbol= fund_code, date= str(date))

# df2 = ak.fund_individual_detail_hold_xq(symbol= "513100", date= str(date))
# df1['代码'] = fund_code
# df2['代码'] = "513870"
# # fund_individual_detail_hold_xq_df = fund_individual_detail_hold_xq_df.T
# df = pd.concat([df1, df2], ignore_index= True)
# print(df)
# df = df.pivot(index='代码', columns='资产类型',values='仓位占比').reset_index()
# display(df)


In [4]:
import pandas as pd

fund_etf_spot_em_df = ak.fund_etf_spot_em()
fund_etf_spot_em_df = fund_etf_spot_em_df[fund_etf_spot_em_df['代码'].isin(stock_list)]
fund_etf_spot_em_df = fund_etf_spot_em_df[['代码','名称','基金折价率','涨跌幅','流通市值','总市值']]
fund_etf_spot_em_df['流通市值'] = fund_etf_spot_em_df['流通市值']/100000000
fund_etf_spot_em_df['总市值'] = fund_etf_spot_em_df['总市值']/100000000
fund_etf_spot_em_df.sort_values(by=['基金折价率','流通市值','基金折价率'], inplace= True, ascending=False)

hold_xq_all = pd.DataFrame()

# 遍历每个基金代码，获取其持仓数据
for fund_code in fund_etf_spot_em_df['代码']:
    try:
        # 获取持仓数据
        fund_individual_detail_hold_xq_df = ak.fund_individual_detail_hold_xq(symbol= fund_code, date= "2024")
        fund_individual_detail_hold_xq_df['代码'] = fund_code
        hold_xq_all = pd.concat([hold_xq_all, fund_individual_detail_hold_xq_df], ignore_index=True)
    except Exception as e:
        print(f"获取基金 {fund_code} 的持仓数据时出错: {e}")
        
hold_xq_all = hold_xq_all.pivot(index='代码', columns='资产类型',values='仓位占比').reset_index()
hold_xq_all = hold_xq_all[['代码','股票','现金','其他']]
# 通过代码关联，并合并
fund_etf_spot_em_df = pd.merge(fund_etf_spot_em_df,hold_xq_all, on='代码', how='left')
display(fund_etf_spot_em_df)

# 初始化一个空的 DataFrame 来存储所有基金的持仓数据
all_fund_holdings = pd.DataFrame()

# 初始化一个空的 DataFrame 来存储最终结果
final_df = pd.DataFrame()

# 遍历每个基金代码，获取其持仓数据
for fund_code in fund_etf_spot_em_df['代码']:
    try:
        # 获取持仓数据
        fund_holdings = ak.fund_portfolio_hold_em(symbol=fund_code, date=2024)
        fund_holdings['基金代码'] = fund_code  # 添加一列以便后续合并
        fund_holdings['基金名称'] = fund_etf_spot_em_df[fund_etf_spot_em_df["代码"] == fund_code]['名称'].iloc[0]
        # display(all_fund_holdings[])
        # 将“基金代码”和“基金名称”列放到最左边
        cols = ['基金代码', '基金名称'] + [col for col in fund_holdings.columns if col not in ['基金代码', '基金名称']]
        fund_holdings = fund_holdings[cols]
        display(fund_holdings)
    except Exception as e:
        print(f"获取基金 {fund_code} 的持仓数据时出错: {e}")

,代码,名称,基金折价率,涨跌幅,流通市值,总市值,股票,现金,其他
0,159659,纳斯达克100ETF,-1.36,0.81,24.291211,24.291211,98.24,2.16,0.91
1,513390,纳指100ETF,-1.38,0.91,17.696314,17.696314,94.04,4.24,1.82
2,513870,纳指ETF富国,-1.40,0.76,9.184204,9.184204,98.03,12.98,1.11
3,513110,纳斯达克100ETF,-1.41,0.82,20.332115,20.332115,93.52,9.55,3.07
4,513290,纳指生物科技ETF,-1.46,0.71,13.845520,13.845520,89.18,0.72,10.43
5,159660,纳指100ETF,-1.49,0.86,14.805865,14.805865,95.30,7.43,1.78
6,159513,纳斯达克100指数ETF,-1.57,0.94,56.116187,56.116187,80.57,13.57,6.67
7,159501,纳斯达克指数ETF,-1.59,0.92,50.578204,50.578204,94.24,6.46,1.84
8,159696,纳指ETF易方达,-1.72,0.93,10.443479,10.443479,93.21,5.77,1.87
9,159632,纳斯达克ETF,-2.60,1.06,81.082852,81.082852,89.90,8.45,2.05


,基金代码,基金名称,序号,股票代码,股票名称,占净值比例,持股数,持仓市值,季度
0,159659,纳斯达克100ETF,1,MSFT,微软,8.57,1.65,5259.92,2024年2季度股票投资明细
1,159659,纳斯达克100ETF,2,AAPL,苹果,8.33,3.41,5114.22,2024年2季度股票投资明细
2,159659,纳斯达克100ETF,3,NVDA,英伟达,7.84,5.47,4812.16,2024年2季度股票投资明细
3,159659,纳斯达克100ETF,4,AMZN,亚马逊,5.19,2.31,3184.49,2024年2季度股票投资明细
4,159659,纳斯达克100ETF,5,AVGO,博通,4.96,0.27,3042.51,2024年2季度股票投资明细
...,...,...,...,...,...,...,...,...,...
107,159659,纳斯达克100ETF,108,AVGO,博通,4.41,0.22,2077.30,2024年1季度股票投资明细
108,159659,纳斯达克100ETF,109,GOOGL,谷歌-A,2.47,1.09,1164.87,2024年1季度股票投资明细
109,159659,纳斯达克100ETF,110,GOOG,谷歌-C,2.40,1.05,1130.95,2024年1季度股票投资明细
110,159659,纳斯达克100ETF,111,TSLA,特斯拉,2.36,0.89,1109.91,2024年1季度股票投资明细


,基金代码,基金名称,序号,股票代码,股票名称,占净值比例,持股数,持仓市值,季度
0,513390,纳指100ETF,1,MSFT,微软,8.27,4.12,13116.52,2024年2季度股票投资明细
1,513390,纳指100ETF,2,AAPL,苹果,8.04,8.50,12752.29,2024年2季度股票投资明细
2,513390,纳指100ETF,3,NVDA,英伟达,7.56,13.63,11998.97,2024年2季度股票投资明细
3,513390,纳指100ETF,4,AMZN,亚马逊,5.00,5.77,7940.70,2024年2季度股票投资明细
4,513390,纳指100ETF,5,AVGO,博通,4.78,0.66,7588.53,2024年2季度股票投资明细
...,...,...,...,...,...,...,...,...,...
108,513390,纳指100ETF,109,GOOGL,谷歌-A,2.29,2.56,2736.23,2024年1季度股票投资明细
109,513390,纳指100ETF,110,GOOG,谷歌-C,2.22,2.46,2656.31,2024年1季度股票投资明细
110,513390,纳指100ETF,111,TSLA,特斯拉,2.18,2.09,2606.96,2024年1季度股票投资明细
111,513390,纳指100ETF,112,COST,开市客,2.16,0.50,2582.89,2024年1季度股票投资明细


,基金代码,基金名称,序号,股票代码,股票名称,占净值比例,持股数,持仓市值,季度
0,513870,纳指ETF富国,1,MSFT,微软,8.56,0.75,2379.76,2024年2季度股票投资明细
1,513870,纳指ETF富国,2,AAPL,苹果,8.32,1.54,2313.71,2024年2季度股票投资明细
2,513870,纳指ETF富国,3,NVDA,英伟达,7.83,2.47,2177.08,2024年2季度股票投资明细
3,513870,纳指ETF富国,4,AMZN,亚马逊,5.18,1.05,1440.75,2024年2季度股票投资明细
4,513870,纳指ETF富国,5,AVGO,博通,4.95,0.12,1376.51,2024年2季度股票投资明细
...,...,...,...,...,...,...,...,...,...
108,513870,纳指ETF富国,109,GOOGL,谷歌-A,2.47,0.94,1007.45,2024年1季度股票投资明细
109,513870,纳指ETF富国,110,GOOG,谷歌-C,2.40,0.91,977.98,2024年1季度股票投资明细
110,513870,纳指ETF富国,111,TSLA,特斯拉,2.36,0.77,959.87,2024年1季度股票投资明细
111,513870,纳指ETF富国,112,COST,开市客,2.34,0.18,951.24,2024年1季度股票投资明细


,基金代码,基金名称,序号,股票代码,股票名称,占净值比例,持股数,持仓市值,季度
0,513110,纳斯达克100ETF,1,MSFT,微软,8.33,2.51,7995.16,2024年2季度股票投资明细
1,513110,纳斯达克100ETF,2,AAPL,苹果,8.10,5.18,7773.32,2024年2季度股票投资明细
2,513110,纳斯达克100ETF,3,NVDA,英伟达,7.62,8.31,7314.21,2024年2季度股票投资明细
3,513110,纳斯达克100ETF,4,AMZN,亚马逊,5.04,3.51,4840.36,2024年2季度股票投资明细
4,513110,纳斯达克100ETF,5,AVGO,博通,4.82,0.40,4624.97,2024年2季度股票投资明细
...,...,...,...,...,...,...,...,...,...
107,513110,纳斯达克100ETF,108,AVGO,博通,4.16,0.37,3437.08,2024年1季度股票投资明细
108,513110,纳斯达克100ETF,109,GOOGL,谷歌-A,2.34,1.80,1927.31,2024年1季度股票投资明细
109,513110,纳斯达克100ETF,110,GOOG,谷歌-C,2.27,1.73,1871.05,2024年1季度股票投资明细
110,513110,纳斯达克100ETF,111,TSLA,特斯拉,2.23,1.47,1836.30,2024年1季度股票投资明细


...

,基金代码,基金名称,序号,股票代码,股票名称,占净值比例,持股数,持仓市值,季度
0,159632,纳斯达克ETF,1,MSFT,微软,7.91,20.21,64359.78,2024年2季度股票投资明细
1,159632,纳斯达克ETF,2,AAPL,苹果,7.69,41.69,62576.23,2024年2季度股票投资明细
2,159632,纳斯达克ETF,3,NVDA,英伟达,7.23,66.87,58878.87,2024年2季度股票投资明细
3,159632,纳斯达克ETF,4,AMZN,亚马逊,4.79,28.30,38976.29,2024年2季度股票投资明细
4,159632,纳斯达克ETF,5,AVGO,博通,4.57,3.25,37216.05,2024年2季度股票投资明细
...,...,...,...,...,...,...,...,...,...
107,159632,纳斯达克ETF,108,AVGO,博通,4.04,3.27,30758.84,2024年1季度股票投资明细
108,159632,纳斯达克ETF,109,GOOGL,谷歌-A,2.26,16.10,17244.51,2024年1季度股票投资明细
109,159632,纳斯达克ETF,110,GOOG,谷歌-C,2.20,15.50,16746.36,2024年1季度股票投资明细
110,159632,纳斯达克ETF,111,TSLA,特斯拉,2.16,13.18,16438.74,2024年1季度股票投资明细


,基金代码,基金名称,序号,股票代码,股票名称,占净值比例,持股数,持仓市值,季度
0,159941,纳指ETF,1,MSFT,微软,8.22,54.68,174176.66,2024年2季度股票投资明细
1,159941,纳指ETF,2,AAPL,苹果,7.90,111.56,167460.66,2024年2季度股票投资明细
2,159941,纳指ETF,3,NVDA,英伟达,7.79,187.61,165183.78,2024年2季度股票投资明细
3,159941,纳指ETF,4,AMZN,亚马逊,4.94,76.01,104680.95,2024年2季度股票投资明细
4,159941,纳指ETF,5,AVGO,博通,4.42,8.19,93714.65,2024年2季度股票投资明细
...,...,...,...,...,...,...,...,...,...
109,159941,纳指ETF,110,GOOGL,谷歌-A,2.37,43.36,46431.56,2024年1季度股票投资明细
110,159941,纳指ETF,111,GOOG,谷歌-C,2.31,42.00,45375.20,2024年1季度股票投资明细
111,159941,纳指ETF,112,TSLA,特斯拉,2.21,34.72,43307.44,2024年1季度股票投资明细
112,159941,纳指ETF,113,COST,开市客,2.11,7.95,41325.74,2024年1季度股票投资明细


,基金代码,基金名称,序号,股票代码,股票名称,占净值比例,持股数,持仓市值,季度
0,513100,纳指ETF,1,MSFT,微软,7.99,32.77,104396.42,2024年2季度股票投资明细
1,513100,纳指ETF,2,AAPL,苹果,7.77,67.62,101498.82,2024年2季度股票投资明细
2,513100,纳指ETF,3,NVDA,英伟达,7.31,108.47,95503.00,2024年2季度股票投资明细
3,513100,纳指ETF,4,AMZN,亚马逊,4.84,45.89,63202.33,2024年2季度股票投资明细
4,513100,纳指ETF,5,AVGO,博通,4.62,5.28,60394.70,2024年2季度股票投资明细
...,...,...,...,...,...,...,...,...,...
107,513100,纳指ETF,108,AVGO,博通,4.02,5.09,47834.23,2024年1季度股票投资明细
108,513100,纳指ETF,109,GOOGL,谷歌-A,2.25,25.04,26817.47,2024年1季度股票投资明细
109,513100,纳指ETF,110,GOOG,谷歌-C,2.19,24.10,26034.65,2024年1季度股票投资明细
110,513100,纳指ETF,111,TSLA,特斯拉,2.15,20.49,25551.13,2024年1季度股票投资明细


,基金代码,基金名称,序号,股票代码,股票名称,占净值比例,持股数,持仓市值,季度
0,513300,纳斯达克ETF,1,MSFT,微软,8.48,11.56,37548.62,2024年2季度股票投资明细
1,513300,纳斯达克ETF,2,AAPL,苹果,8.24,23.85,36506.37,2024年2季度股票投资明细
2,513300,纳斯达克ETF,3,NVDA,英伟达,7.75,38.27,34349.86,2024年2季度股票投资明细
3,513300,纳斯达克ETF,4,AMZN,亚马逊,5.13,16.19,22732.09,2024年2季度股票投资明细
4,513300,纳斯达克ETF,5,AVGO,博通,4.90,1.86,21722.58,2024年2季度股票投资明细
...,...,...,...,...,...,...,...,...,...
108,513300,纳斯达克ETF,109,GOOGL,谷歌-A,2.43,9.10,9920.29,2024年1季度股票投资明细
109,513300,纳斯达克ETF,110,GOOG,谷歌-C,2.36,8.76,9630.74,2024年1季度股票投资明细
110,513300,纳斯达克ETF,111,TSLA,特斯拉,2.32,7.44,9451.81,2024年1季度股票投资明细
111,513300,纳斯达克ETF,112,COST,开市客,2.30,1.77,9365.23,2024年1季度股票投资明细


,基金代码,基金名称,序号,股票代码,股票名称,占净值比例,持股数,持仓市值,季度
0,159509,纳指科技ETF,1,NVDA,英伟达,12.96,127.78,112505.89,2024年2季度股票投资明细
1,159509,纳指科技ETF,2,AAPL,苹果,12.60,72.87,109380.97,2024年2季度股票投资明细
2,159509,纳指科技ETF,3,MSFT,微软,12.38,33.75,107493.83,2024年2季度股票投资明细
3,159509,纳指科技ETF,4,META,Meta Platforms Inc-A,7.27,17.57,63121.19,2024年2季度股票投资明细
4,159509,纳指科技ETF,5,AVGO,博通,7.19,5.46,62424.56,2024年2季度股票投资明细
5,159509,纳指科技ETF,6,GOOGL,谷歌-A,6.18,41.34,53670.05,2024年2季度股票投资明细
6,159509,纳指科技ETF,7,GOOG,谷歌-C,5.95,39.53,51679.67,2024年2季度股票投资明细
7,159509,纳指科技ETF,8,AMD,超威半导体,2.53,19.03,21997.21,2024年2季度股票投资明细
8,159509,纳指科技ETF,9,ADBE,奥多比,2.41,5.27,20880.94,2024年2季度股票投资明细
9,159509,纳指科技ETF,10,QCOM,高通,2.15,13.14,18649.74,2024年2季度股票投资明细
